In [8]:
#! ls
! ls cifar-10-batches-py

batches.meta  data_batch_2  data_batch_4  readme.html
data_batch_1  data_batch_3  data_batch_5  test_batch


In [5]:
! tar -xvzf main/cifar-10-python.tar.gz

cifar-10-batches-py/
cifar-10-batches-py/data_batch_4
cifar-10-batches-py/readme.html
cifar-10-batches-py/test_batch
cifar-10-batches-py/data_batch_3
cifar-10-batches-py/batches.meta
cifar-10-batches-py/data_batch_2
cifar-10-batches-py/data_batch_5
cifar-10-batches-py/data_batch_1


In [96]:
import numpy as np
import pickle
import collections
import tensorflow as tf
import matplotlib.pyplot as plt

% matplotlib inline

In [151]:
def loaddata(fpath):
    with open(fpath, 'rb') as f:
        d = pickle.load(f, encoding='bytes')
        d_decoded = {}
        for k, v in d.items():
            d_decoded[k.decode('utf8')] = v
        d = d_decoded
    return d['data'], d['labels']

In [184]:
d1, l1 = loaddata('cifar-10-batches-py/data_batch_1')
d2, l2 = loaddata('cifar-10-batches-py/data_batch_2')
d3, l3 = loaddata('cifar-10-batches-py/data_batch_2')
d4, l4 = loaddata('cifar-10-batches-py/data_batch_2')
d5, l5 = loaddata('cifar-10-batches-py/data_batch_2')

In [185]:
img = np.vstack([d1, d2, d3, d4, d5])
img = img.reshape(-1, 32, 32, 3).astype(np.float32)
del d1, d2, d3, d4, d5
lab = np.concatenate([np.array(l1), np.array(l2), np.array(l3), np.array(l4), np.array(l5)])
del l1, l2, l3, l4, l5

1. airplane  
2. automobile  
3. bird  
4. cat  
5. deer  
6. dog  
7. frog  
8. horse  
9. ship  
10. truck 

In [99]:
class getdata(object):
    
    def __init__(self, images, labels, one_hot = True):
        assert images.shape[0] == labels.shape[0]
        self.num_examples = images.shape[0]
        assert images.shape[3] == 3
        self.images = images
        self.one_hot = one_hot
        self.labels = labels
        if not one_hot:
            self.labels = onehot(labels)
        self.epoch = 0
        self.startofepoch = 0
        
    def nextbatch(self, batch_size):
        start = self.startofepoch
        end = start + batch_size
        
        if end > self.num_examples:
            self.epoch += 1
            
            perm = np.arange(self.num_examples)
            np.random.shuffle(perm)
            
            self.images = self.images[perm]
            self.labels = self.labels[perm]
            
            start = 0
            end = batch_size
            
        self.startofepoch = end
        return self.images[start:end], self.labels[start:end]

In [76]:
Datasets = collections.namedtuple('Datasets', ['train', 'test'])

def createdata(images, labels):
    perm = np.arange(images.shape[0])
    np.random.shuffle(perm)
    train_images = images[perm]
    train_labels = labels[perm]
    validation_images = images[int(0.7*train_images.shape[0]):]
    validation_labels = labels[int(0.7*train_labels.shape[0]):]
    train_images = images[:int(0.7*train_images.shape[0])]
    train_labels = labels[:int(0.7*train_labels.shape[0])]
    
    train = getdata(train_images, train_labels, one_hot = True)
    validation = getdata(validation_images, validation_labels, one_hot = True)
    #test = getdata(test_images, test_labels, one_hot = True)
    
    return Datasets(train=train, test=validation)

In [187]:
data = createdata(img, lab)

In [177]:
tf.logging.set_verbosity(tf.logging.INFO)

In [178]:
def model(features, labels, mode):

    input_layer = tf.reshape(features["x"], [-1, 32, 32, 3])
    conv1 = tf.layers.conv2d(inputs=input_layer,
        filters=32,
        kernel_size=[5, 5],
        padding="same",
        activation=tf.nn.relu,
        name = "Conv1")

    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2, name = "Pool1")
    conv2 = tf.layers.conv2d(inputs=pool1,
        filters=64,
        kernel_size=[5, 5],
        padding="same",
        activation=tf.nn.relu,
        name = "Conv2")

    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2, name = "Pool2")
    pool2_flat = tf.reshape(pool2, [-1, 8 * 8 * 64])
    dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu, name = "Fc1")

    dropout = tf.layers.dropout(inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

    logits = tf.layers.dense(inputs=dropout, units=10, name = "out")

    predictions = {
        "classes": tf.argmax(input=logits, axis=1, name = "Prediction"),
        "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }

    eval_metric_ops = {
        "accuracy": tf.metrics.accuracy(labels=labels, predictions=predictions["classes"], name = "Accuracy_value")}

    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    # Calculate Loss (for both TRAIN and EVAL modes)
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(loss=loss, global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {"accuracy": tf.metrics.accuracy(labels=labels, predictions=predictions["classes"], name = "Accuracy_value")}
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [179]:
classifier = tf.estimator.Estimator(model_fn=model, model_dir="tboard/try1")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_tf_random_seed': None, '_task_type': 'worker', '_model_dir': 'tboard/try1', '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_num_ps_replicas': 0, '_log_step_count_steps': 100, '_save_checkpoints_secs': 600, '_num_worker_replicas': 1, '_session_config': None, '_service': None, '_master': '', '_is_chief': True, '_keep_checkpoint_every_n_hours': 10000, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f9b27c14e10>, '_keep_checkpoint_max': 5, '_task_id': 0}


In [190]:
train_fn = tf.estimator.inputs.numpy_input_fn(x={"x": data.train.images}, 
                                                    y=data.train.labels, 
                                                    batch_size=100,
                                                    num_epochs=None,
                                                    shuffle=True)

In [188]:
type(data.train.images[1,1,1,1])

numpy.float32

In [191]:
classifier.train(input_fn=train_fn,steps=20000)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into tboard/try1/model.ckpt.
INFO:tensorflow:loss = 50.8401, step = 1
INFO:tensorflow:global_step/sec: 1.77446
INFO:tensorflow:loss = 2.27509, step = 101 (56.140 sec)
INFO:tensorflow:global_step/sec: 1.83463
INFO:tensorflow:loss = 2.21173, step = 201 (54.455 sec)
INFO:tensorflow:global_step/sec: 1.78698
INFO:tensorflow:loss = 2.24979, step = 301 (55.961 sec)
INFO:tensorflow:global_step/sec: 1.80082
INFO:tensorflow:loss = 2.15891, step = 401 (55.530 sec)
INFO:tensorflow:global_step/sec: 1.62159
INFO:tensorflow:loss = 2.10446, step = 501 (61.670 sec)
INFO:tensorflow:global_step/sec: 1.77151
INFO:tensorflow:loss = 2.15974, step = 601 (56.487 sec)
INFO:tensorflow:global_step/sec: 1.807
INFO:tensorflow:loss = 2.08132, step = 701 (55.300 sec)
INFO:tensorflow:global_step/sec: 1.83963
INFO:tensorflow:loss = 2.03748, step = 801 (54.359 sec)
INFO:tensorflow:global_step/sec: 1.75388
INFO:tensorflow:loss = 2.0727,

KeyboardInterrupt: 